# 📥 Download Dataset

**What it does:**  
- Imports **KaggleHub**, a library that simplifies downloading Kaggle datasets directly in Colab.  
- Downloads the dataset using the given **Kaggle ID**.  
- Prints the folder location where the dataset is saved.  

**Optional steps:**  
1. **Use the official Kaggle API** – more reliable for larger or complex datasets.  
2. **Mount Google Drive** – useful if you want to save the dataset directly to your Drive:  
   ```python
   from google.colab import drive
   drive.mount('/content/drive')


In [ ]:
import kagglehub
path = kagglehub.dataset_download("sehriyarmemmedli/glasses-vs-noglasses-dataset")
print("Path to dataset files:", path)

100%|██████████| 2.61G/2.61G [00:29<00:00, 95.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sehriyarmemmedli/glasses-vs-noglasses-dataset/versions/1


# 📚 Import Libraries

**What it does:**  
- Imports everything needed for:  
  - **Data augmentation** (`ImageDataGenerator`)  
  - **Model building** (`Sequential`)  
  - **CNN layers** (`Conv2D`, `MaxPooling2D`, etc.)  
  - **Single-image prediction** (`load_img`, `img_to_array`)  


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import load_img, img_to_array

# 📂 Define Dataset Paths

**Why it's done:**  
- You must tell TensorFlow where images are stored.  
- `output_folder` might be used to create a balanced version of the dataset.  

**Optional steps:**  
1. **Auto-detect folders**  
2. **Combine all images into a DataFrame**  
   ```python
   import pandas as


In [ ]:
# Original dataset path (from kagglehub cache)
dataset_path = "/root/.cache/kagglehub/datasets/sehriyarmemmedli/glasses-vs-noglasses-dataset/versions/1"

# Output folder for 25k balanced dataset
output_dir = "dataset_25k"
os.makedirs(output_dir, exist_ok=True)

# Classes
classes = ["with_glasses", "without_glasses"]
splits = ["train", "val", "test"]

# Collect all images
all_images = {c: [] for c in classes}
for split in splits:
    for cls in classes:
        folder = os.path.join(dataset_path, split, cls)
        if os.path.exists(folder):
            for f in os.listdir(folder):
                if f.lower().endswith((".jpg", ".png", ".jpeg")):
                    all_images[cls].append(os.path.join(folder, f))

# Sample 25k images, balanced
TARGET = 25000
TARGET_PER_CLASS = TARGET // 2
sampled = {cls: random.sample(all_images[cls], TARGET_PER_CLASS) for cls in classes}
print("Sampled per class:", {cls: len(sampled[cls]) for cls in classes})

# Split 80% train, 20% test and copy
TRAIN_RATIO = 0.8
for cls in classes:
    random.shuffle(sampled[cls])
    train_count = int(len(sampled[cls]) * TRAIN_RATIO)

    train_imgs = sampled[cls][:train_count]
    test_imgs  = sampled[cls][train_count:]

    train_dir = os.path.join(output_dir, "train", cls)
    test_dir  = os.path.join(output_dir, "test", cls)
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, train_dir)
    for img in test_imgs:
        shutil.copy(img, test_dir)

print("Done! New dataset saved to:", output_dir)

Sampled per class: {'with_glasses': 12500, 'without_glasses': 12500}
Done! New dataset saved to: dataset_25k


# 🎨 Data Preparation & Augmentation

**What it does:**  
- Normalizes images to **0–1 range**.  
- Applies **random transformations** to increase diversity.  

**Why it’s done:**  
- **Augmentation** helps prevent overfitting.  
- **Normalization** makes training stable.  

**Benefits:**  
- Makes your model **robust**.  
- Works great when the dataset is **small**.  


In [ ]:
# ===============================
# 3️⃣ Prepare Data Generators
# ===============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    'dataset_25k/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    'dataset_25k/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# 🏗 Build the CNN Model

**What it does:**  
- Extracts features using **convolution**.  
- Reduces spatial size using **pooling**.  
- Flattens to feed into **Dense layers**.  
- Uses **sigmoid** for binary output.  

**Optional steps:**  
1. **Transfer Learning** (highly recommended)  
   - Achieves **95%+ accuracy** easily  
   - Works well with **small datasets**  
   - Commonly used in **professional projects**  


In [ ]:
# ===============================
# 4️⃣ Build the CNN Model
# ===============================
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# 🏃 Train the Model

**What it does:**  
- Runs **training loop**.  
- Updates **weights**.  
- Evaluates after each **epoch**.  

**Optional steps:**  
1. **Callback for early stopping**  
   - Prevents **overfitting**  
   - Saves **training time**  
2. **Save model checkpoints**  
   - Keeps the **best version** only  


In [ ]:
# ===============================
# 5️⃣ Train the Model
# ===============================
history = model.fit(
    train_set,
    steps_per_epoch=len(train_set),
    validation_data=test_set,
    validation_steps=len(test_set),
    epochs=10
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 191s 302ms/step - accuracy: 0.8526 - loss: 0.3120 - val_accuracy: 0.9758 - val_loss: 0.0669
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 186s 297ms/step - accuracy: 0.9628 - loss: 0.1091 - val_accuracy: 0.9736 - val_loss: 0.0718
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 185s 296ms/step - accuracy: 0.9744 - loss: 0.0743 - val_accuracy: 0.9794 - val_loss: 0.0687
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 298ms/step - accuracy: 0.9795 - loss: 0.0624 - val_accuracy: 0.9876 - val_loss: 0.0396
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 182s 292ms/step - accuracy: 0.9809 - loss: 0.0596 - val_accuracy: 0.9890 - val_loss: 0.0340
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 188s 301ms/step - accuracy: 0.9793 - loss: 0.0605 - val_accuracy: 0.9886 - val_loss: 0.0393
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 189s 302ms/step - accuracy: 0.9803 - loss: 0.0570 - val_accuracy: 0.9876 - val_loss: 0.0408
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 189s 302ms/step - accuracy: 0.9816 -

In [ ]:
# ===============================
# 6️⃣ Save the Model
# ===============================
model.save("glasses_classifier.keras")
print("Model saved as glasses_classifier.keras")

Model saved as glasses_classifier.keras


In [ ]:
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

def upload_and_predict_colab():
    # Upload image from local
    uploaded = files.upload()

    # Get the filename of uploaded image
    img_path = list(uploaded.keys())[0]

    # Load and preprocess image
    img = load_img(img_path, target_size=(64,64))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predict
    prediction = model.predict(img_array)[0][0]
    # Correct mapping based on class_indices
    if prediction >= 0.5:
        print("Prediction: without_glasses 🚫")
    else:
        print("Prediction: with_glasses 😎")

In [ ]:
upload_and_predict_colab()

Saving with_glasses.jpg to with_glasses.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: with_glasses 😎


In [ ]:
upload_and_predict_colab()

Saving without glasses.jpg to without glasses.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: without_glasses 🚫


In [ ]:
upload_and_predict_colab()

Saving with glasses.jpg to with glasses.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Prediction: with_glasses 😎


In [ ]:
upload_and_predict_colab()

Saving with glasses2.jpg to with glasses2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Prediction: with_glasses 😎


In [ ]:
upload_and_predict_colab()

Saving without glassesV.avif to without glassesV.avif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: without_glasses 🚫


In [ ]:
# Download to local
files.download("glasses_classifier.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>